## Вопросы для самопроверки:
* что такое обучени с подкреплением (reinforcement learning)?
* что такое среда?
* что такое агент?
* что такое награда, какая она может быть?

#  Taxi (а на самом деле FrozenLake8x8)

Такси учится долго и печально, и застревает в локальном максимуме вида "пассажира не брать и не высаживать, просто ездить". А FrozenLake8x8 прекрасно учится, так что пусть будет он

In [35]:
import gym

#create a single game instance
# env = gym.make("Taxi-v2").env
env = gym.make("FrozenLake8x8-v0")
# env = gym.make("FrozenLake-v0")

#start new game
env.reset();

In [25]:
action2id = {
    'south': 0,
    'north': 1,
    'east': 2,
    'west': 3,
    'pickup': 4,
    'dropoff': 5,
}

In [36]:
# display the game state
s, r, done, _ = env.step(2)
print(r)
env.render()

0.0
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG


### Gym interface

The three main methods of an environment are
* __reset()__ - reset environment to initial state, _return first observation_
* __render()__ - show current environment state (a more colorful version :) )
* __step(a)__ - commit action __a__ and return (new observation, reward, is done, info)
 * _new observation_ - an observation right after commiting the action __a__
 * _reward_ - a number representing your reward for commiting action __a__
 * _is done_ - True if the MDP has just finished, False if still in progress
 * _info_ - some auxilary stuff about what just happened. Ignore it for now

In [37]:
print("initial observation code:", env.reset())
print('printing observation:')
env.render()
print("observations:", env.observation_space, 'n=', env.observation_space.n)
print("actions:", env.action_space, 'n=', env.action_space.n)

initial observation code: 0
printing observation:

SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
observations: Discrete(64) n= 64
actions: Discrete(4) n= 4


In [38]:
print("taking action 2 (right)")
new_obs, reward, is_done, info = env.step(2)
print("new observation code:", new_obs)
print("reward:", reward)
print("is game over?:", is_done)
print("printing new state:")
env.render()
print(info)

taking action 2 (right)
new observation code: 0
reward: 0.0
is game over?: False
printing new state:
  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
{'prob': 0.3333333333333333}


### Baseline: random search

In [39]:
import numpy as np

n_states = env.observation_space.n
n_actions = env.action_space.n

print(n_states, n_actions)

def get_random_policy():
    """
    Build a numpy array representing agent policy.
    This array must have one element per each of 16 environment states.
    Element must be an integer from 0 to 3, representing action
    to take from that state.
    """
    return np.random.randint(0, n_actions, size=n_states)

64 4


In [40]:
np.random.seed(501)
policies = [get_random_policy() for i in range(10**4)]
assert all([len(p) == n_states for p in policies]), 'policy length should always be 16'
assert np.min(policies) == 0, 'minimal action id should be 0'
assert np.max(policies) == n_actions-1, 'maximal action id should match n_actions-1'
action_probas = np.unique(policies, return_counts=True)[-1] /10**4. /n_states
print("Action frequencies over 10^4 samples:",action_probas)
assert np.allclose(action_probas, [1. / n_actions] * n_actions, atol=0.05), "The policies aren't uniformly random (maybe it's just an extremely bad luck)"
print("Seems fine!")

Action frequencies over 10^4 samples: [ 0.25012031  0.25034844  0.25007656  0.24945469]
Seems fine!


### Let's evaluate!
* Implement a simple function that runs one game and returns the total reward

In [41]:
def sample_reward(env, policy, t_max=100):
    """
    Interact with an environment, return sum of all rewards.
    If game doesn't end on t_max (e.g. agent walks into a wall), 
    force end the game and return whatever reward you got so far.
    Tip: see signature of env.step(...) method above.
    """
    s = env.reset()
    total_reward = 0
    
    for i in range(t_max):
        action = policy[s]
        s, r, done, _ = env.step(action)
        total_reward += r
        
        if done:
            break
    
    return total_reward

In [42]:
env.reset()

0

In [43]:
sample_reward(env, get_random_policy())

0.0

In [44]:
print("generating 10^3 sessions...")
rewards = [sample_reward(env, get_random_policy()) for _ in range(10 ** 3)]
assert all([type(r) in (int, float) for r in rewards]), 'sample_reward must return a single number'
print("Looks good!")
print(min(rewards), max(rewards))

generating 10^3 sessions...
Looks good!
0.0 1.0


In [55]:
def evaluate(policy, n_times=100):
    """Run several evaluations and average the score the policy gets."""
    rewards = [sample_reward(env, policy) for _ in range(n_times)]
    return float(np.mean(rewards))
        

### Main loop

In [46]:
from tqdm import tqdm

best_policy = None
best_score = -float('inf')


for i in tqdm(range(100)):
    policy = get_random_policy()
    score = evaluate(policy)
    if score > best_score:
        best_score = score
        best_policy = policy
        print("New best score:", score)

  2%|▏         | 2/100 [00:00<00:07, 12.29it/s]

New best score: 0.0


  5%|▌         | 5/100 [00:00<00:09, 10.53it/s]

New best score: 0.01


100%|██████████| 100/100 [00:10<00:00, 10.08it/s]


# Part II Genetic algorithm 

The next task is to devise some more effecient way to perform policy search.
We'll do that with a bare-bones evolutionary algorithm.
[unless you're feeling masochistic and wish to do something entirely different which is bonus points if it works]

In [47]:
from scipy.stats import bernoulli

p = (1, 2)



In [48]:
from scipy.stats import bernoulli


def crossover(policy1, policy2, p=0.5):
    """
    for each state, with probability p take action from policy1, else policy2
    """
    policies = [policy1, policy2]
    policy = np.zeros(len(policy1), dtype=int)
    choises = bernoulli.rvs(p, size=len(policy1))
    
    for i, choise in enumerate(choises):
        policy[i] = policies[choise][i]
    return np.array(policy)

In [49]:
def mutation(policy, p=0.1):
    """
    for each state, with probability p replace action with random action
    Tip: mutation can be written as crossover with random policy
    """
    return crossover(policy, get_random_policy(), p)
    

In [50]:
np.random.seed(501)
policies = [crossover(get_random_policy(), get_random_policy()) 
            for i in range(10 ** 3)]

assert all([len(p) == n_states for p in policies]), 'policy length should always be 16'
assert np.min(policies) == 0, 'minimal action id should be 0'
assert np.max(policies) == n_actions - 1, 'maximal action id should be n_actions-1'

assert any([np.mean(crossover(np.zeros(n_states), np.ones(n_states))) not in (0, 1)
               for _ in range(100)]), "Make sure your crossover changes each action independently"
print("Seems fine!")

Seems fine!


In [56]:
from multiprocessing import Pool

executor = Pool(3)

def get_scores(pool):
    return list(executor.map(evaluate, pool))


n_epochs = 100  # how many cycles to make
pool_size = 100  # how many policies to maintain
n_crossovers = 50  # how many crossovers to make on each step
n_mutations = 50  # how many mutations to make on each tick


Process ForkPoolWorker-1:
Process ForkPoolWorker-3:
Process ForkPoolWorker-2:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 342, in get
    res = self._reader.recv_bytes()
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = s

In [57]:
%%time
print("initializing...")
pool = [get_random_policy() for _ in range(pool_size)] # spawn a list of pool_size random policies
pool_scores = get_scores(pool)

initializing...
CPU times: user 3.04 ms, sys: 5.63 ms, total: 8.66 ms
Wall time: 4.74 s


In [58]:
assert type(pool) == type(pool_scores) == list
assert len(pool) == len(pool_scores) == pool_size
assert all([type(score) in (float, int) for score in pool_scores])

In [59]:
import time 


# main loop
for epoch in range(n_epochs):
    print("Epoch %s:" % epoch)
    start = time.time()
    
    c_indices = [np.random.choice(pool_size, 2, False) for _ in range(n_crossovers)]
    m_indices = np.random.choice(pool_size, n_mutations, False)
    
    crossovered = [crossover(pool[i1], pool[i2]) for i1, i2 in c_indices]
    
    mutated = [mutation(pool[i]) for i in m_indices]
    
    assert type(crossovered) == type(mutated) == list
    
    # add new policies to the pool
    pool = pool + crossovered + mutated
    pool_scores = get_scores(pool)
    
    # select pool_size best policies
    selected_indices = np.argsort(pool_scores)[-pool_size:]
    pool = [pool[i] for i in selected_indices]
    pool_scores = [pool_scores[i] for i in selected_indices]

    # print the best policy so far (last in ascending score order)
    print("best score: ", pool_scores[-1])
    print("time for epoch: ", time.time() - start)

Epoch 0:
best score:  0.07
time for epoch:  9.266743898391724
Epoch 1:
best score:  0.05
time for epoch:  10.274072885513306
Epoch 2:
best score:  0.06
time for epoch:  10.66352105140686
Epoch 3:
best score:  0.06
time for epoch:  10.23140811920166
Epoch 4:
best score:  0.08
time for epoch:  9.232786655426025
Epoch 5:
best score:  0.06
time for epoch:  9.242061614990234
Epoch 6:
best score:  0.09
time for epoch:  9.122787475585938
Epoch 7:
best score:  0.17
time for epoch:  9.992283344268799
Epoch 8:
best score:  0.11
time for epoch:  10.21208381652832
Epoch 9:
best score:  0.24
time for epoch:  10.801365852355957
Epoch 10:
best score:  0.27
time for epoch:  11.262792587280273
Epoch 11:
best score:  0.31
time for epoch:  11.99470829963684
Epoch 12:
best score:  0.3
time for epoch:  13.170142650604248
Epoch 13:
best score:  0.35
time for epoch:  13.374397993087769
Epoch 14:
best score:  0.4
time for epoch:  13.556926965713501
Epoch 15:
best score:  0.45
time for epoch:  13.7665324211120

In [60]:
print(pool_scores[-1])

0.73


In [70]:
from IPython.display import clear_output
from time import sleep

trained = pool[-1]

score = 0

for i in range(20):
    s = env.reset()
    total_reward = 0

    for i in range(10000):
        action = trained[s]
        s, r, done, _ = env.step(action)
        total_reward += r

        clear_output(wait=True)
        env.render()
        sleep(0.1)
        if done:
            break

    print(total_reward)
    sleep(1)
    score += total_reward
print(score / 20)

  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
1.0
0.75
